In [90]:
import pandas as pd
from gensim import utils
from gensim.models import FastText
from razdel import tokenize


In [98]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
address_test = test['address']
address = train['address']

In [92]:
class MyIter:
    def __iter__(self):
        for i in range(len(address)) :
            tokens = list(tokenize(address[i]))
            yield [_.text for _ in tokens]    

model = FastText(vector_size=64, window=3, min_count=1)
model.build_vocab(corpus_iterable=MyIter())
total_examples = model4.corpus_count
model.train(corpus_iterable=MyIter(), total_examples=total_examples, epochs=1)

(1756305, 4434733)

In [102]:
model.wv.similarity(address_test[0], address[0])

0.9018372